In [1]:
from ipywidgets import Button, Output, VBox
from ipyfilechooser import FileChooser
import os
import openai
import json
import warnings
import datetime

# Ignore all warnings
warnings.filterwarnings("ignore")

# Code that generates warnings
openai.api_type = "azure"
openai.api_base = "https://acceuubyteopenai.openai.azure.com/" 
openai.api_version = "2023-05-15"
openai.api_key = "8ddd9f33112a43b0894c7ad0a0aa6263"
def run_code(btn):
    from IPython.display import display, HTML

    message1 = "<h1><b>Splitting the input code into smaller chunks</b></h1>"
    message2 = "<h1><b>Transforming each chunk code into embedding</b></h1>"
    message3 = "<h1><b>Storing the code chunk embeddings in vector store</b></h1>"

    messages = message1 + "<br>" + message2 + "<br>" + message3
    display(HTML(messages))
    # Get the file paths of the two chosen files
    file1_path = file1_chooser.selected
    file2_path = file2_chooser.selected
    file3_path= file3_chooser.selected
    # Print the file paths of the two files
    def read_code_file(file_path):
        with open(file_path, 'r') as file:
            code = file.read()
        return code

    
    main_code = read_code_file(file1_path)
    functions_code = read_code_file(file2_path)

    Explanation = {'Document Number and Name': '<Give the document number and document name present in code>',
    'Plant Name': '<Give plant name present in code>',
    'Squad Team': '<Squad team with designation with line numbers>',
    'Model Objective': '<Explain the main logic of entire code and explain>',
    'Input features': '<Input Features with Explanation>',
    'Output features': '<List output features>',
    'Model Used': '<Give which models are used and explain how it predict value in code>',
    'Explanation of steps': '<Give main steps or logic involved in the code1 in seperate lines>',
    'Modules used' : '<Explain user defined functions used in both codes in seperate lines>'}



    prompt = f"""You are a code explanation bot.Give output as per below format.
    Output Format:
    {json.dumps(Explanation, indent=4)}

    Code1:
    {main_code}

    Code2:
    {functions_code}

    Output:

    """
    response = openai.Completion.create(
        engine="gpt-35-turbo",
        prompt=prompt,
        max_tokens=2000,
        temperature=0,
        n=1,
        stop=None)

    # Extract the differences from the API response
    output = response.choices[0].text.strip()

    output = response.choices[0].text.replace('<|im_end|>','')

    print(output)
    from docx import Document
    from docx.shared import Pt


    data = json.loads(output)

    # Create a new Word document
    doc = Document()
    for key,value in data.items():
        # Create a new paragraph for each key-value pair
        key_paragraph = doc.add_paragraph()
        # Add the key and value to the corresponding paragraphs
        key_paragraph.add_run(key + ":").bold = True
        #value_paragraph.add_run(str(value))
        for step in value.split('\n'):

            p = doc.add_paragraph(step)
            p.style.paragraph_format.line_spacing = 1

    # Save the document
    doc.save("my_document_new.docx")
    temp_file = 'Code_explanation.docx'
    doc.save(temp_file)

    # Open the file in Jupyter
    os.system('start ' + temp_file)

   
    
file1_chooser = FileChooser()
file1_chooser.title = 'Choose Code1'
file2_chooser = FileChooser()
file2_chooser.title = 'Choose Code2'
file3_chooser = FileChooser()
file3_chooser.title = 'Choose Code3'


# Create a Submit button
submit_btn = Button(description='Generate Code Explanation')
submit_btn.layout.width = '300px'  # set the button color to white
submit_btn.style.font_weight = 'bold'  # make the label bold
submit_btn.style.font_size = '16px'  # set the font size to 16 pixels

submit_btn.on_click(run_code)

# Create an output widget to display the results
output = Output()

# Combine the widgets into a vertical box layout
vbox = VBox([file1_chooser, file2_chooser,file3_chooser, submit_btn, output])

# Display the widgets
display(vbox)